In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from opti_recruit.data import get_data, clean_data
import opti_recruit.feature_engineering as fe
import opti_recruit.get_team_features as gtf
import pandas as pd

In [ ]:
! pwd

In [ ]:
! cd

In [ ]:
! ls

In [3]:
dfs = get_data()
dfs.keys()

dict_keys([17, 18, 19, 20, 21, 22])

In [4]:
df22 = dfs[22]
df21 = dfs[21]

In [6]:
df22_new = fe.add_features(df22)
df21_new = fe.add_features(df21)

In [7]:
df22_new.columns

Index(['sofifa_id', 'short_name', 'player_positions', 'overall', 'potential',
       'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg',
       'club_team_id', 'club_name', 'league_name', 'league_level',
       'club_position', 'club_joined', 'club_contract_valid_until',
       'nationality_name', 'nation_team_id', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'body_type',
       'release_clause_eur', 'player_tags', 'player_traits', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina'

In [24]:
listing=list(dfs)
print(listing)

[17, 18, 19, 20, 21, 22]


In [89]:
teams_avg = ['df22_team','df21_team']
team_pos = ['df22_team_pos','df21_team_pos']
team_start = ['df22_team_start','df21_team_start']
frames = [df22_new,df21_new]

d_avg = {teams_avg[i]: frames[i] for i in range(0,2)}
d_avg_pos = {team_pos[i]: frames[i] for i in range(0,2)}
d_avg_start = {team_start[i]: frames[i] for i in range(0,2)}


In [51]:
d_avg

{'df22_team':        sofifa_id          short_name player_positions  overall  potential  \
 0         158023            L. Messi       RW, ST, CF       93         93   
 1         188545      R. Lewandowski               ST       92         92   
 2          20801   Cristiano Ronaldo           ST, LW       91         91   
 3         190871           Neymar Jr          LW, CAM       91         91   
 4         192985        K. De Bruyne          CM, CAM       91         91   
 ...          ...                 ...              ...      ...        ...   
 19234     261962           Song Defu              CDM       47         52   
 19235     262040           C. Porter               CM       47         59   
 19236     262760            N. Logue               CM       47         55   
 19237     262820           L. Rudden               ST       47         60   
 19238     264540  E. Lalchhanchhuaha              CAM       47         60   
 
          value_eur  wage_eur  age  height_cm  we

In [63]:
for df in d_avg.keys():
    vars()[df] = d_avg[df].groupby(['club_name'])[['value_eur','overall','potential']].mean() \
        .reset_index() \
        .add_suffix('_mean') \
        .rename(columns={"club_name_mean": "club_name"})

In [64]:
df21_team

,club_name,value_eur_mean,overall_mean,potential_mean
0,1. FC Heidenheim 1846,1.349333e+06,66.266667,71.433333
1,1. FC Kaiserslautern,8.403571e+05,62.785714,68.821429
2,1. FC Köln,4.252500e+06,70.266667,76.700000
3,1. FC Magdeburg,6.175926e+05,62.074074,68.000000
4,1. FC Nürnberg,1.790500e+06,66.833333,72.866667
...,...,...,...,...
676,Örebro SK,5.523077e+05,60.884615,65.692308
677,Östersunds FK,5.520370e+05,60.814815,66.259259
678,İstanbul Başakşehir FK,4.541667e+06,71.433333,74.466667
679,İttifak Holding Konyaspor,1.405769e+06,68.230769,70.307692


In [68]:
for df in d_avg_pos.keys():
    vars()[df]  = d_avg_pos[df].groupby(['club_name','player_pos'])[['value_eur','overall','potential']] \
        .mean() \
        .reset_index() \
        .pivot(index='club_name',columns='player_pos',values=['value_eur','overall','potential'])
    vars()[df].columns = list(map("_".join, vars()[df].columns))
    vars()[df] = vars()[df].rename(columns={"club_name_": "club_name"}).reset_index()

In [69]:
df21_team_pos

,club_name,value_eur_ATT,value_eur_DEF,value_eur_GK,value_eur_MID,overall_ATT,overall_DEF,overall_GK,overall_MID,potential_ATT,potential_DEF,potential_GK,potential_MID
0,1. FC Heidenheim 1846,1.287500e+06,1.789286e+06,8.825000e+05,1.284615e+06,65.666667,68.571429,63.250000,66.230769,70.333333,73.142857,71.000000,71.153846
1,1. FC Kaiserslautern,9.062500e+05,8.225000e+05,6.750000e+05,8.553571e+05,65.000000,62.625000,59.500000,62.714286,67.500000,68.125000,70.000000,69.428571
2,1. FC Köln,4.216667e+06,3.352273e+06,4.516667e+06,4.961538e+06,70.000000,68.272727,71.000000,71.846154,74.666667,76.000000,76.666667,77.769231
3,1. FC Magdeburg,5.750000e+05,6.750000e+05,5.833333e+05,5.875000e+05,62.750000,62.700000,60.333333,61.700000,65.750000,68.600000,69.666667,67.800000
4,1. FC Nürnberg,1.437500e+06,1.442500e+06,1.110000e+06,2.622500e+06,66.500000,66.200000,67.250000,67.500000,71.333333,72.400000,72.000000,74.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Örebro SK,5.333333e+05,4.762500e+05,3.750000e+05,6.916667e+05,61.333333,61.000000,57.000000,61.777778,64.500000,64.500000,63.000000,68.444444
677,Östersunds FK,5.750000e+05,6.083333e+05,2.783333e+05,5.844444e+05,61.333333,62.333333,59.666667,60.666667,67.333333,67.000000,63.000000,66.500000
678,İstanbul Başakşehir FK,4.995000e+06,3.252083e+06,2.566667e+06,6.455000e+06,70.800000,71.333333,71.000000,72.000000,74.400000,74.250000,73.333333,75.100000
679,İttifak Holding Konyaspor,7.875000e+05,1.038636e+06,6.875000e+05,2.015909e+06,65.500000,67.272727,72.000000,69.000000,65.500000,69.000000,72.000000,72.181818


In [91]:
for df in d_avg_start.keys():
    vars()[df]  = d_avg_start[df][d_avg_start[df]['is_bench']==False] \
        .groupby(['club_name','player_pos'])[['value_eur','overall','potential']] \
        .mean() \
        .reset_index() \
        .pivot(index='club_name',columns='player_pos',values=['value_eur','overall','potential'])
    vars()[df].columns = list(map("_".join, vars()[df].columns))
    vars()[df] = vars()[df].rename(columns={"club_name_": "club_name"}) \
        .add_suffix('_start').reset_index()

In [92]:
df21_team_start

,club_name,value_eur_ATT_start,value_eur_DEF_start,value_eur_GK_start,value_eur_MID_start,overall_ATT_start,overall_DEF_start,overall_GK_start,overall_MID_start,potential_ATT_start,potential_DEF_start,potential_GK_start,potential_MID_start
0,1. FC Heidenheim 1846,1200000.0,2.175000e+06,2100000.0,1.555000e+06,67.0,71.500000,73.0,70.000000,68.0,73.500000,73.0,70.600000
1,1. FC Kaiserslautern,937500.0,1.250000e+06,1100000.0,1.100000e+06,64.5,68.000000,64.0,65.400000,68.5,69.666667,72.0,69.400000
2,1. FC Köln,7500000.0,5.700000e+06,10500000.0,7.100000e+06,76.0,72.750000,77.0,73.000000,76.0,77.750000,80.0,80.200000
3,1. FC Magdeburg,650000.0,8.687500e+05,1000000.0,5.562500e+05,65.0,64.750000,66.0,63.750000,65.0,69.000000,71.0,66.250000
4,1. FC Nürnberg,1200000.0,2.118750e+06,2000000.0,3.575000e+06,68.0,68.750000,72.0,68.800000,68.0,74.000000,74.0,76.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Örebro SK,662500.0,6.750000e+05,925000.0,9.875000e+05,64.0,64.000000,68.0,65.500000,64.0,66.750000,69.0,69.250000
677,Östersunds FK,NaN,6.083333e+05,275000.0,7.285714e+05,NaN,62.333333,66.0,61.857143,NaN,67.000000,66.0,68.142857
678,İstanbul Başakşehir FK,1000000.0,4.675000e+06,5500000.0,1.106000e+07,72.0,76.250000,77.0,75.600000,72.0,76.250000,77.0,76.800000
679,İttifak Holding Konyaspor,1000000.0,1.462500e+06,625000.0,2.480000e+06,67.0,70.500000,75.0,71.200000,67.0,70.500000,75.0,72.000000


In [94]:
d_avg_pos.keys()

dict_keys(['df22_team_pos', 'df21_team_pos'])

In [100]:
team_avg,team_avg_pos,avg_pos_start = gtf.get_team_metrics()
team_avg.keys()

dict_keys(['df22_team', 'df21_team', 'df20_team', 'df19_team'])

In [101]:
team_avg_pos.keys()

dict_keys(['df22_team_pos', 'df21_team_pos', 'df20_team_pos', 'df19_team_pos'])

In [103]:
team_avg_pos['df20_team_pos']

,club_name,value_eur_ATT,value_eur_DEF,value_eur_GK,value_eur_MID,overall_ATT,overall_DEF,overall_GK,overall_MID,potential_ATT,potential_DEF,potential_GK,potential_MID
0,1. FC Heidenheim 1846,6.430000e+05,1.230000e+06,6.300000e+05,1.552500e+06,62.800000,67.300000,62.000000,67.300000,70.000000,71.600000,70.500000,72.800000
1,1. FC Kaiserslautern,7.333333e+05,4.877778e+05,4.483333e+05,6.765000e+05,65.166667,62.777778,60.666667,64.700000,68.500000,69.333333,72.333333,69.900000
2,1. FC Köln,9.333333e+06,4.533333e+06,4.315000e+06,5.340000e+06,77.000000,71.555556,67.250000,71.733333,78.000000,77.222222,74.250000,77.733333
3,1. FC Magdeburg,5.250000e+05,5.535714e+05,4.750000e+05,7.000000e+05,62.750000,64.285714,63.000000,65.181818,70.750000,68.714286,71.500000,70.000000
4,1. FC Nürnberg,2.420455e+06,1.950000e+06,2.030000e+06,2.931250e+06,68.272727,68.375000,69.333333,69.750000,74.454545,74.000000,70.000000,74.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Östersunds FK,4.675000e+05,4.361111e+05,2.433333e+05,4.413636e+05,59.750000,61.666667,59.000000,61.090909,68.000000,68.444444,63.333333,68.909091
664,İstanbul Başakşehir FK,3.900000e+06,3.685000e+06,4.750000e+06,5.830769e+06,72.800000,73.600000,75.000000,73.384615,74.000000,74.500000,75.500000,75.000000
665,İttifak Holding Konyaspor,1.262500e+06,1.246154e+06,2.012500e+06,2.753125e+06,65.750000,68.384615,72.000000,70.250000,72.500000,69.615385,72.000000,73.500000
666,ŁKS Łódź,3.266667e+05,1.850000e+05,1.166667e+05,2.634615e+05,61.666667,57.750000,56.666667,59.153846,61.666667,61.500000,62.333333,64.153846


In [112]:
team_avg.items()

dict_items([('df22_team',                      club_name  value_eur_mean  overall_mean  potential_mean
0        1. FC Heidenheim 1846    1.574074e+06     66.925926       71.185185
1         1. FC Kaiserslautern    7.600000e+05     62.821429       68.142857
2                   1. FC Köln    3.290161e+06     69.161290       75.451613
3              1. FC Magdeburg    6.687500e+05     62.107143       67.500000
4               1. FC Nürnberg    1.281379e+06     65.689655       70.896552
..                         ...             ...           ...             ...
696                  Örebro SK    5.572000e+05     61.440000       65.640000
697              Östersunds FK    5.629167e+05     61.666667       65.833333
698     İstanbul Başakşehir FK    2.963889e+06     70.000000       73.222222
699  İttifak Holding Konyaspor    1.350000e+06     67.750000       69.583333
700              Śląsk Wrocław    7.235714e+05     62.964286       68.428571

[701 rows x 4 columns]), ('df21_team',           

In [113]:
team_avg['df22_team']

,club_name,value_eur_mean,overall_mean,potential_mean
0,1. FC Heidenheim 1846,1.574074e+06,66.925926,71.185185
1,1. FC Kaiserslautern,7.600000e+05,62.821429,68.142857
2,1. FC Köln,3.290161e+06,69.161290,75.451613
3,1. FC Magdeburg,6.687500e+05,62.107143,67.500000
4,1. FC Nürnberg,1.281379e+06,65.689655,70.896552
...,...,...,...,...
696,Örebro SK,5.572000e+05,61.440000,65.640000
697,Östersunds FK,5.629167e+05,61.666667,65.833333
698,İstanbul Başakşehir FK,2.963889e+06,70.000000,73.222222
699,İttifak Holding Konyaspor,1.350000e+06,67.750000,69.583333


In [115]:
df_2022,df_2021,df_2020,df_2019 = gtf.join_team_df()


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,value_eur_GK_start,value_eur_MID_start,overall_ATT_start,overall_DEF_start,overall_GK_start,overall_MID_start,potential_ATT_start,potential_DEF_start,potential_GK_start,potential_MID_start
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
1,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
2,231747,https://sofifa.com/player/231747/kylian-mbappe...,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,194000000.0,230000.0,22,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
3,230621,https://sofifa.com/player/230621/gianluigi-don...,G. Donnarumma,Gianluigi Donnarumma,GK,89,93,119500000.0,110000.0,22,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
4,155862,https://sofifa.com/player/155862/sergio-ramos-...,Sergio Ramos,Sergio Ramos García,CB,88,88,24000000.0,115000.0,35,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
5,193041,https://sofifa.com/player/193041/keylor-navas/...,K. Navas,Keylor Navas Gamboa,GK,88,88,15500000.0,130000.0,34,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
6,183898,https://sofifa.com/player/183898/angel-di-mari...,Á. Di María,Ángel Fabián Di María Hernández,"RW, LW",87,87,49500000.0,160000.0,33,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
7,199556,https://sofifa.com/player/199556/marco-verratt...,M. Verratti,Marco Verratti,"CM, CAM",87,87,79500000.0,155000.0,28,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
8,207865,https://sofifa.com/player/207865/marcos-aoas-c...,Marquinhos,Marcos Aoás Corrêa,"CB, CDM",87,90,90500000.0,135000.0,27,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333
9,235212,https://sofifa.com/player/235212/achraf-hakimi...,A. Hakimi,Achraf Hakimi Mouh,"RB, RWB",85,88,69500000.0,100000.0,22,...,119500000.0,4.716667e+07,91.666667,85.5,89.0,84.333333,93.0,87.0,93.0,84.333333


In [118]:
df_2022.shape

(19239, 137)

In [126]:

import numpy as np 
A=np.array([7,3,5])
B=np.array([3,7,10])



In [127]:

print(A,A.reshape(1,-1))

[7 3 5] [[7 3 5]]


In [128]:
from scipy.spatial import distance
distance.cosine(A.reshape(1,-1),B.reshape(1,-1))

0.1966214394919974

In [129]:
distance.cosine(A,B)

0.1966214394919974

In [132]:
def get_index(x):
    return df_2022[df_2022['short_name']==x].index.tolist()[0]

In [138]:
get_index('Cristiano Ronaldo')

61

In [137]:
df22_new.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,defending_standing_tackle,defending_sliding_tackle,is_bench,potential_diff,att_rate,def_rate,age_bin,prefered_pos,player_pos,new_nationality
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,...,35,24,False,0,Medium,Low,30-34,RW,ATT,Argentina
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,...,42,19,False,0,High,Medium,30-34,ST,ATT,Poland
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,...,32,24,False,0,High,Low,Over 34,ST,ATT,Portugal
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,...,32,29,False,0,High,Medium,25-29,LW,ATT,Brazil
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,...,65,53,False,0,High,High,30-34,CM,MID,Others


In [141]:

from sklearn.metrics.pairwise import cosine_similarity 
# def recommend_cosine(player):
#     index = get_index(player)

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = df_2022.select_dtypes(include=numerics).fillna(0)

similarities = cosine_similarity(newdf)

In [152]:
newdf.loc[61,:]

sofifa_id                 20801.00
overall                      91.00
potential                    91.00
value_eur              45000000.00
wage_eur                 270000.00
                          ...     
overall_MID_start            84.20
potential_ATT_start          91.00
potential_DEF_start          86.75
potential_GK_start           84.00
potential_MID_start          87.40
Name: 61, Length: 87, dtype: float64

In [147]:
playerIndex = similarities[61,:].argsort()[-16:][::-1][1:]

In [149]:
playerIndex = similarities[61,:].argsort()[-16:][::-1][1:]
for i in playerIndex:
        print(df_2022.loc[i,'short_name'])

H. Maguire
L. Shaw
A. Wan-Bissaka
D. van de Beek
D. Henderson
S. Uçan
G. Nkoudou
C. Larin
A. Martial
P. Köpke
N. Geyrhofer
J. Geis
M. Sarkaria
K. Fischer
A. Nordin


In [158]:
from sklearn.neighbors import NearestNeighbors

reco = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(newdf)
players_recommended = reco.kneighbors(newdf)[1]



In [157]:
print('Here are 10 players similar to Cristiano Ronaldo:' '\n')
for i in players_recommended[61][1:]:
    print(df_2022.iloc[i]['short_name'])

Here are 10 players similar to Cristiano Ronaldo:

H. Maguire
L. Shaw
A. Wan-Bissaka
D. van de Beek
D. Henderson
A. Martial
M. Greenwood
S. McTominay
De Gea
